# CSV dialect detection with CleverCSV

**Author**: [Gertjan van den Burg](https://gertjan.dev)

In this note we'll show some examples of using CleverCSV, a package for 
handling messy CSV files. We'll start with a motivating example and then show 
some other files where CleverCSV shines. CleverCSV was developed as part of a 
research project on automating data wrangling. It achieves an accuracy of 97% 
on over 9300 real-world CSV files and improves the accuracy on messy files by 
21% over standard tools.

Handy links:

 - [Paper on arXiv](https://arxiv.org/abs/1811.11242)
 - [CleverCSV on GitHub](https://github.com/alan-turing-institute/CleverCSV)
 - [CleverCSV on PyPI](https://pypi.org/project/clevercsv/)
 - [Reproducible Research Repo](https://github.com/alan-turing-institute/CSV_Wrangling/)

## IMDB Movie data

Alice is a data scientist who would like to analyse the movie ratings on IMDB 
for movies of different genres. She found [a dataset shared by a user on 
Kaggle](https://www.kaggle.com/orgesleka/imdbmovies) that contains information 
of over 14,000 movies. Great! 

The data is stored in a CSV file, which is a very common data format for 
sharing tabular data. The first few lines of the file look like this:

```
fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,nrOfWins,nrOfNominations,nrOfPhotos,nrOfNewsArticles,nrOfUserReviews,nrOfGenre,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,FilmNoir,GameShow,History,Horror,Music,Musical,Mystery,News,RealityTV,Romance,SciFi,Short,Sport,TalkShow,Thriller,War,Western
titles01/tt0012349,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550,3240,1921,video.movie,1,0,19,96,85,3,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
titles01/tt0015864,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319,5700,1925,video.movie,2,1,35,110,122,3,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
titles01/tt0017136,tt0017136,Metropolis (1927),metropolis,http://www.imdb.com/title/tt0017136/,8.4,81007,9180,1927,video.movie,3,4,67,428,376,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
titles01/tt0017925,tt0017925,Der General (1926),der general,http://www.imdb.com/title/tt0017925/,8.3,37521,6420,1926,video.movie,1,1,53,123,219,3,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
titles01/tt0021749,tt0021749,Lichter der Großstadt (1931),lichter der gro stadt,http://www.imdb.com/title/tt0021749/,8.7,70057,5220,1931,video.movie,2,0,38,187,186,3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
```

Let's load it with Pandas!



In [1]:
%xmode Minimal
import pandas as pd
df = pd.read_csv('./data/imdb.csv')

Exception reporting mode: Minimal


ParserError: Error tokenizing data. C error: Expected 44 fields in line 66, saw 46



Oh, that doesn't work. Maybe there's something wrong with the file? Let's try 
opening it with the Python CSV reader:



In [2]:
import csv
with open('./data/imdb.csv', 'r', newline='') as fid:
    dialect = csv.Sniffer().sniff(fid.read())
    print("Detected delimiter = %r, quotechar = %r" % (dialect.delimiter, dialect.quotechar))
    fid.seek(0)
    reader = csv.reader(fid, dialect=dialect)
    rows = list(reader)

print("Loaded %i rows." % len(rows))

Detected delimiter = ' ', quotechar = "'"
Loaded 13928 rows.



Huh, that's strange, Python thinks the *space* is the delimiter and loads 
13928 rows, but the file should contain 14,762 rows according to the 
documentation.  What's going on here?

It turns out that on the 65th line of the file, there's a movie with the title 
``Dr. Seltsam\, oder wie ich lernte\, die Bombe zu lieben (1964)`` (the German 
version of Dr. Strangelove).  The title has commas in it, that are escaped 
using the ``\`` character!  Why are CSV files so hard? 😑

**CleverCSV to the rescue!**

CleverCSV detects the dialect of CSV files much more accurately than existing 
approaches, and it is therefore robust against these kinds of format 
variations. It even has a wrapper that works with DataFrames!



In [3]:
from ccsv.wrappers import csv2df

df = csv2df('./data/imdb.csv')
df

,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,...,News,RealityTV,Romance,SciFi,Short,Sport,TalkShow,Thriller,War,Western
0,titles01/tt0012349,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550.0,3240.0,1921.0,video.movie,...,0,0,0,0,0,0,0,0,0,0
1,titles01/tt0015864,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319.0,5700.0,1925.0,video.movie,...,0,0,0,0,0,0,0,0,0,0
2,titles01/tt0017136,tt0017136,Metropolis (1927),metropolis,http://www.imdb.com/title/tt0017136/,8.4,81007.0,9180.0,1927.0,video.movie,...,0,0,0,1,0,0,0,0,0,0
3,titles01/tt0017925,tt0017925,Der General (1926),der general,http://www.imdb.com/title/tt0017925/,8.3,37521.0,6420.0,1926.0,video.movie,...,0,0,0,0,0,0,0,0,0,0
4,titles01/tt0021749,tt0021749,Lichter der Großstadt (1931),lichter der gro stadt,http://www.imdb.com/title/tt0021749/,8.7,70057.0,5220.0,1931.0,video.movie,...,0,0,1,0,0,0,0,0,0,0
5,titles01/tt0022100,tt0022100,M (1931),m,http://www.imdb.com/title/tt0022100/,8.5,73726.0,7020.0,1931.0,video.movie,...,0,0,0,0,0,0,0,1,0,0
6,titles01/tt0025316,tt0025316,Es geschah in einer Nacht (1934),es geschah in einer nacht,http://www.imdb.com/title/tt0025316/,8.3,46503.0,6300.0,1934.0,video.movie,...,0,0,1,0,0,0,0,0,0,0
7,titles01/tt0027977,tt0027977,Moderne Zeiten (1936),moderne zeiten,http://www.imdb.com/title/tt0027977/,8.6,90847.0,5220.0,1936.0,video.movie,...,0,0,0,0,0,0,0,0,0,0
8,titles01/tt0031381,tt0031381,Vom Winde verweht (1939),vom winde verweht,http://www.imdb.com/title/tt0031381/,8.2,160414.0,14280.0,1939.0,video.movie,...,0,0,1,0,0,0,0,0,1,0
9,titles01/tt0031679,tt0031679,Mr. Smith geht nach Washington (1939),mr smith geht nach washington,http://www.imdb.com/title/tt0031679/,8.4,58169.0,7740.0,1939.0,video.movie,...,0,0,0,0,0,0,0,0,0,0



🎉

## Other Examples

We'll compare CleverCSV to the built-in Python CSV module and to Pandas and 
show how these are not as robust as CleverCSV. Note that Pandas always uses 
the comma as separator, unless it is forced to autodetect the dialect. These 
files are of course selected for this tutorial, because it wouldn't be very 
interesting to show files where all methods are correct.

The example CSV files all come from MIT-licensed GitHub repositories, and the 
URLs point directly to the source files.

First we'll define some functions for easy comparisons.



In [4]:
import csv
import ccsv
import io
import requests
import pandas as pd

from termcolor import colored
from IPython.display import display

def page(url):
    """ Get the content of a webpage using requests, assuming UTF-8 encoding """
    page = requests.get(url)
    content = page.content.decode('utf-8')
    return content

def head(content, num=10):
    """ Preview a CSV file """
    print('--- File Preview ---')
    for i, line in enumerate(io.StringIO(content, newline=None)):
        print(line, end='')
        if i == num - 1:
            break
    print('\n---')

def sniff_url(content):
    """ Utility to run the python Sniffer on a CSV file at a URL """
    try:
        dialect = csv.Sniffer().sniff(content)
        print("CSV Sniffer detected: delimiter = %r, quotechar = %r" % (dialect.delimiter,
                                                                        dialect.quotechar))
    except csv.Error as err:
        print(colored("No result from the Python CSV Sniffer", "red"))
        print(colored("Error was: %s" % err, "red"))

def detect_url(content, verbose=True):
    """ Utility to run the CleverCSV detector on a CSV file at a URL """
    # We have designed CleverCSV to be a drop-in replacement for the CSV module
    try:
        dialect = ccsv.Sniffer().sniff(content, verbose=verbose)
        print("CleverCSV detected: delimiter = %r, quotechar = %r" % (dialect.delimiter, 
                                                                      dialect.quotechar))
    except ccsv.Error:
        print(colored("No result from CleverCSV", "red"))

def pandas_url(content):
    """ Wrapper around pandas.read_csv(). """
    buf = io.StringIO(content)
    print(
        "Pandas uses: delimiter = %r, quotechar = %r"
        % (',', '"')
    )
    try:
        df = pd.read_csv(buf)
        display(df.head())
    except pd.errors.ParserError:
        print(colored("ParserError from pandas.", "red"))


def compare(url, verbose=False, n_preview=10):
    content = page(url)
    head(content, num=n_preview)
    print("\n1. Running Python Sniffer")
    sniff_url(content)
    print("\n2. Running Pandas")
    pandas_url(content)
    print("\n3. Running CleverCSV")
    detect_url(content, verbose=verbose)



## Example 1: No output from Python Sniffer

The first file we'll look at is a simple CSV file that uses the semicolon as delimiter. 




In [5]:
compare("https://raw.githubusercontent.com/grezesf/Research/17b1e829d1d4b8954661270bd8b099e74bb45ce7/Reservoirs/Task0_Replication/code/preprocessing/factors.csv")

--- File Preview ---
log(E);de_log(E);dede_log(E);cepstrum;de_cepstrum;dede_cepstrum
0.27;1.77;4.97;0.1;0.61;1.75;
1.75;1.25;1.00;1.25;0.50;0.25;
---

1. Running Python Sniffer
No result from the Python CSV Sniffer
Error was: Could not determine delimiter

2. Running Pandas
Pandas uses: delimiter = ',', quotechar = '"'


,log(E);de_log(E);dede_log(E);cepstrum;de_cepstrum;dede_cepstrum
0,0.27;1.77;4.97;0.1;0.61;1.75;
1,1.75;1.25;1.00;1.25;0.50;0.25;



3. Running CleverCSV
CleverCSV detected: delimiter = ';', quotechar = ''



As we can see, the Python CSV sniffer fails on this one, even though the 
formatting doesn't seem to be that uncommon. CleverCSV handles this file 
correctly.

## Example 2: Incorrect output from Python Sniffer

The next example is quite a long file with a lot of potential delimiters. In 
total, CleverCSV considers 180 different dialects on this file and determines 
the best dialect by computing a pattern score and a type score. The pattern 
score is related to how many cells we have per row given a certain dialect, 
and the type score reflects whether the cells in the parsed file have known 
data types (such as integer, date, string, etc.). 

If you want to see the output of CleverCSV while it runs the detection, you 
can set ``verbose=True`` in the next line.



In [6]:
compare("https://raw.githubusercontent.com/agh-glk/pyconpl2013-nlp/37f6f50a45fc31c1a5ad25010fff681a8ce645b8/gsm.csv", verbose=False)

--- File Preview ---
name,price,state
NOWY DOTYKOWY LG T375 Wi-Fi DUAL SIM + GRATISY FV,238.0,new
NOWY SAMSUNG GALAXY NOTE 2 N7100 WHITE FV 23%,1799.0,new
NOKIA 3510 12miesięcy  GWARANCJI   Dostawa 24h,79.0,new
NOKIA 206 DUAL SIM GWAR. 2 LATA FV23% OKAZJA!,199.0,new
SAMSUNG B2100i SOLID BEZ SIML. +ETUI 24GW PL FV23%,298.0,new
Polski SAMSUNG S5300 GALAXY POCKET Gwar 24M-ce,318.0,new
Polski SAMSUNG S5610 Gwar 24M-ce KRAKÓW 5Mpx Foto,298.0,new
BIAŁY SAMSUNG I8190 GALAXY S III Mini Gwar 24M-ce,898.0,new
= SONY XPERIA E C1505 White/Biały!!! Wrocław =,475.0,new

---

1. Running Python Sniffer
CSV Sniffer detected: delimiter = '!', quotechar = '"'

2. Running Pandas
Pandas uses: delimiter = ',', quotechar = '"'


,name,price,state
0,NOWY DOTYKOWY LG T375 Wi-Fi DUAL SIM + GRATISY FV,238.0,new
1,NOWY SAMSUNG GALAXY NOTE 2 N7100 WHITE FV 23%,1799.0,new
2,NOKIA 3510 12miesięcy GWARANCJI Dostawa 24h,79.0,new
3,NOKIA 206 DUAL SIM GWAR. 2 LATA FV23% OKAZJA!,199.0,new
4,SAMSUNG B2100i SOLID BEZ SIML. +ETUI 24GW PL F...,298.0,new



3. Running CleverCSV
CleverCSV detected: delimiter = ',', quotechar = '"'



Note that CleverCSV is a bit slower than the Python Sniffer. This is the focus 
of our ongoing development efforts and is also affected by running Python 
through Jupyter. But let's not forget that at least CleverCSV is correct!

## Conclusion

Below are some more examples, but I think you'll get the idea by now. 
CleverCSV is much more robust against messy CSV files and is an easy to use 
drop-in replacement for the Python csv module. Just replace ``import csv`` by 
``import ccsv`` in your code!

We're still working on adding some more features to CleverCSV and speeding up 
and improving the dialect detection algorithm. One of the novel features that 
we added is a ``clevercsv`` command line executable with the following 
commands:

- ``detect`` to run dialect detection directly from the command line

- ``view`` to open a CSV file in a table viewer after automatic detection of 
  the dialect

- ``standardize`` to convert a CSV file in a messy format to the standard CSV 
  format


We hope you find CleverCSV useful! If you encounter any issues or files where 
CleverCSV fails, please leave a comment on GitHub.

## Further Examples




In [7]:
# No result from Python (note that this file says it uses "|" as separator, but actually uses ","!)
compare("https://raw.githubusercontent.com/queq/just-stuff/c1b8714664cc674e1fc685bd957eac548d636a43/pov/TopFixed/build/project_r_pad.csv", n_preview=30)

--- File Preview ---
#Release 14.4 - par P.49d (lin)
#Copyright (c) 1995-2012 Xilinx, Inc.  All rights reserved.

#Thu Jun 20 07:23:42 2013

#
## NOTE: This file is designed to be imported into a spreadsheet program
# such as Microsoft Excel for viewing, printing and sorting. The |
# character is used as the data field separator. This file is also designed
# to support parsing.
#
#INPUT FILE:       project.ncd
#OUTPUT FILE:      project_r_pad.csv
#PART TYPE:        xc3s500e
#SPEED GRADE:      -5
#PACKAGE:          fg320
#
# Pinout by Pin Number:
# 
# -----,-----,-----,-----,-----,-----,-----,-----,-----,-----,-----,-----,-----,-----,-----,
Pin Number,Signal Name,Pin Usage,Pin Name,Direction,IO Standard,IO Bank Number,Drive (mA),Slew Rate,Termination,IOB Delay,Voltage,Constraint,IO Register,Signal Integrity,
A1,,,GND,,,,,,,,,,,,
A2,,,TDI,,,,,,,,,,,,
A3,,IBUF,IP,UNUSED,,0,,,,,,,,,
A4,,DIFFM,IO_L24P_0,UNUSED,,0,,,,,,,,,
A5,,DIFFMI,IP_L22P_0,UNUSED,,0,,,,,,,,,
A6,,DIFFS,IO_L20N_0,UNUSED,,0

In [8]:
# Python says '\r' (carriage return) is the delimiter!
compare("https://raw.githubusercontent.com/HAYASAKA-Ryosuke/TodenGraphDay/8f052219d037edabebd488e5f6dc2ddbe8367dc1/juyo-j.csv")

--- File Preview ---
2013/12/12 23:20 UPDATE
ピーク時供給力(万kW),時間帯,供給力情報更新日,供給力情報更新時刻
4849,17:00〜18:00,12/12,8:30

予想最大電力(万kW),時間帯,予想最大電力情報更新日,予想最大電力情報更新時刻
4210,17:00〜18:00,12/12,8:30

DATE,TIME,当日実績(万kW),予測値（万kW)
2013/12/12,0:00,3098,0
2013/12/12,1:00,2948,0

---

1. Running Python Sniffer
CSV Sniffer detected: delimiter = '\r', quotechar = '"'

2. Running Pandas
Pandas uses: delimiter = ',', quotechar = '"'


,,,2013/12/12 23:20 UPDATE
ピーク時供給力(万kW),時間帯,供給力情報更新日,供給力情報更新時刻
4849,17:00〜18:00,12/12,8:30
予想最大電力(万kW),時間帯,予想最大電力情報更新日,予想最大電力情報更新時刻
4210,17:00〜18:00,12/12,8:30
DATE,TIME,当日実績(万kW),予測値（万kW)



3. Running CleverCSV
CleverCSV detected: delimiter = ',', quotechar = ''


In [9]:
# No result from Python csv
compare("https://raw.githubusercontent.com/philipmcg/minecraft-service-windows/774892ff0c27a76b6db20ba3750149c19b7a3351/MinecraftService/MinecraftService/gcsv_sample.csv")

--- File Preview ---
// comment
 // another comment

~test_name,row_name,col1,col2
row1,r1c1,r1c2
row2,r2c1,r2c2
---

1. Running Python Sniffer
No result from the Python CSV Sniffer
Error was: Could not determine delimiter

2. Running Pandas
Pandas uses: delimiter = ',', quotechar = '"'
ParserError from pandas.

3. Running CleverCSV
CleverCSV detected: delimiter = ',', quotechar = ''


In [10]:
# Incorrect delimiter from Python csv
compare("https://raw.githubusercontent.com/OptimusGitEtna/RestSymf/635e4ad8a288cde64b306126c986213de71a4f4a/Python-3.4.2/Doc/tools/sphinxext/susp-ignored.csv")

--- File Preview ---
c-api/arg,,:ref,"PyArg_ParseTuple(args, ""O|O:ref"", &object, &callback)"
c-api/list,,:high,list[low:high]
c-api/sequence,,:i2,del o[i1:i2]
c-api/sequence,,:i2,o[i1:i2]
c-api/unicode,,:end,str[start:end]
c-api/unicode,,:start,unicode[start:start+length]
distutils/examples,267,`,This is the description of the ``foobar`` package.
distutils/setupscript,,::,
extending/embedding,,:numargs,"if(!PyArg_ParseTuple(args, "":numargs""))"
extending/extending,,:myfunction,"PyArg_ParseTuple(args, ""D:myfunction"", &c);"

---

1. Running Python Sniffer
CSV Sniffer detected: delimiter = ' ', quotechar = '"'

2. Running Pandas
Pandas uses: delimiter = ',', quotechar = '"'


,c-api/arg,Unnamed: 1,:ref,"PyArg_ParseTuple(args, ""O|O:ref"", &object, &callback)"
0,c-api/list,NaN,:high,list[low:high]
1,c-api/sequence,NaN,:i2,del o[i1:i2]
2,c-api/sequence,NaN,:i2,o[i1:i2]
3,c-api/unicode,NaN,:end,str[start:end]
4,c-api/unicode,NaN,:start,unicode[start:start+length]



3. Running CleverCSV
CleverCSV detected: delimiter = ',', quotechar = '"'
